# LSTM-arithmetic

## Dataset
- [Arithmetic dataset](https://drive.google.com/file/d/1cMuL3hF9jefka9RyF4gEBIGGeFGZYHE-/view?usp=sharing)

In [2]:
# ! pip install seaborn
# ! pip install opencc
# ! pip install -U scikit-learn

import numpy as np
import pandas as pd
import torch
import torch.nn
import torch.nn.utils.rnn
import torch.utils.data
import matplotlib.pyplot as plt
import seaborn as sns
import opencc
import os
from sklearn.model_selection import train_test_split

data_path = './data'

In [3]:
df_train = pd.read_csv(os.path.join(data_path, 'arithmetic_train.csv'))
df_eval = pd.read_csv(os.path.join(data_path, 'arithmetic_eval.csv'))
df_train.head()

,Unnamed: 0,src,tgt
0,2285313,14*(43+20)=,882
1,317061,(6+1)*5=,35
2,718770,13+32+29=,74
3,170195,31*(3-11)=,-248
4,2581417,24*49+1=,1177


In [4]:
# transform the input data to string
df_train['tgt'] = df_train['tgt'].apply(lambda x: str(x))
df_train['src'] = df_train['src'].add(df_train['tgt'])
df_train['len'] = df_train['src'].apply(lambda x: len(x))

df_eval['tgt'] = df_eval['tgt'].apply(lambda x: str(x))

# Build Dictionary
 - The model cannot perform calculations directly with plain text.
 - Convert all text (numbers/symbols) into numerical representations.
 - Special tokens
    - '&lt;pad&gt;'
        - Each sentence within a batch may have different lengths.
        - The length is padded with '&lt;pad&gt;' to match the longest sentence in the batch.
    - '&lt;eos&gt;'
        - Specifies the end of the generated sequence.
        - Without '&lt;eos&gt;', the model will not know when to stop generating.

In [6]:
char_to_id = {}
id_to_char = {}

# write your code here
# Build a dictionary and give every token in the train dataset an id
# The dictionary should contain <eos> and <pad>
# char_to_id is to conver charactors to ids, while id_to_char is the opposite
all_text = ''.join(df_train['src'].tolist())
unique_chars = sorted(set(all_text))
special_tokens = ['<pad>', '<eos>']
current_id = 0

# Add special tokens
for token in special_tokens:
    char_to_id[token] = current_id
    id_to_char[current_id] = token
    current_id += 1

# Add characters from dataset
for ch in unique_chars:
    char_to_id[ch] = current_id
    id_to_char[current_id] = ch
    current_id += 1

vocab_size = len(char_to_id)
print('Vocab size{}'.format(vocab_size))

Vocab size18


# Data Preprocessing
 - The data is processed into the format required for the model's input and output. (End with \<eos\> token)


In [7]:
# Write your code here
def preprocess_data(df_input, char_to_id):
    df = df_input.copy()
    df['len'] = df['src'].apply(lambda x: len(x))
    # Create a new DataFrame to avoid modifying the original one

    # Map src to char IDs
    df['char_id_list'] = df['src'].map(lambda x: [char_to_id.get(ch) for ch in x] + [char_to_id['<eos>']])

    # Map tgt to char IDs
    df['tgt_tokens'] = df['tgt'].map(lambda x: [char_to_id.get(ch) for ch in x])

    # Compute padding length
    df['pad_len'] = df['len'] - df['tgt'].str.len() - 1

    # Create label_id_list by concatenating pad tokens + tgt tokens + <eos>
    df['label_id_list'] = df.apply(
        lambda row: [char_to_id['<pad>']] * row['pad_len'] + row['tgt_tokens'] + [char_to_id['<eos>']] + [char_to_id['<pad>']],
        axis=1
    )

    df = df[['src', 'tgt', 'len', 'char_id_list', 'label_id_list']]
    return df
df_train = preprocess_data(df_train, char_to_id)

df_train.head()

,src,tgt,len,char_id_list,label_id_list
0,14*(43+20)=882,882,14,"[8, 11, 4, 2, 11, 10, 5, 9, 7, 3, 17, 15, 15, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15, 9, 1, 0]"
1,(6+1)*5=35,35,10,"[2, 13, 5, 8, 3, 4, 12, 17, 10, 12, 1]","[0, 0, 0, 0, 0, 0, 0, 10, 12, 1, 0]"
2,13+32+29=74,74,11,"[8, 10, 5, 10, 9, 5, 9, 16, 17, 14, 11, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 14, 11, 1, 0]"
3,31*(3-11)=-248,-248,14,"[10, 8, 4, 2, 10, 6, 8, 8, 3, 17, 6, 9, 11, 15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 9, 11, 15, 1, 0]"
4,24*49+1=1177,1177,12,"[9, 11, 4, 11, 16, 5, 8, 17, 8, 8, 14, 14, 1]","[0, 0, 0, 0, 0, 0, 0, 8, 8, 14, 14, 1, 0]"


# Hyper Parameters

|Hyperparameter|Meaning|Value|
|-|-|-|
|`batch_size`|Number of data samples in a single batch|64|
|`epochs`|Total number of epochs to train|10|
|`embed_dim`|Dimension of the word embeddings|256|
|`hidden_dim`|Dimension of the hidden state in each timestep of the LSTM|256|
|`lr`|Learning Rate|0.001|
|`grad_clip`|To prevent gradient explosion in RNNs, restrict the gradient range|1|

In [9]:
batch_size = 64
epochs = 2
embed_dim = 256
hidden_dim = 256
lr = 0.001
grad_clip = 1

# Data Batching
- Use `torch.utils.data.Dataset` to create a data generation tool called  `dataset`.
- The, use `torch.utils.data.DataLoader` to randomly sample from the `dataset` and group the samples into batches.

- Example: 1+2-3=0
    - Model input: 1 + 2 - 3 = 0
    - Model output: / / / / / 0 &lt;eos&gt;  (the '/' can be replaced with &lt;pad&gt;)
    - The key for the model's output is that the model does not need to predict the next character of the previous part. What matters is that once the model sees '=', it should start generating the answer, which is '0'. After generating the answer, it should also generate&lt;eos&gt;

In [10]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    
    def __len__(self):
        # return the amount of data
        return len(self.sequences)
    
    def __getitem__(self, index):
        # Extract the input data x and the ground truth y from the data
        x = self.sequences.iloc[index]['char_id_list'] # Write your code here
        y = self.sequences.iloc[index]['label_id_list'] # Write your code here
        return x, y

# collate function, used to build dataloader
def collate_fn(batch):
    batch_x = [torch.tensor(data[0]) for data in batch]
    batch_y = [torch.tensor(data[1]) for data in batch]
    batch_x_lens = torch.LongTensor([len(x) for x in batch_x])
    batch_y_lens = torch.LongTensor([len(y) for y in batch_y])
    
    # Pad the input sequence
    pad_batch_x = torch.nn.utils.rnn.pad_sequence(batch_x,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    pad_batch_y = torch.nn.utils.rnn.pad_sequence(batch_y,
                                                  batch_first=True,
                                                  padding_value=char_to_id['<pad>'])
    
    return pad_batch_x, pad_batch_y, batch_x_lens, batch_y_lens

In [12]:
ds_train = Dataset(df_train[['char_id_list', 'label_id_list']])

In [ ]:
# Build dataloader of train set and eval set, collate_fn is the collate function
# Write your code here
from torch.utils.data import DataLoader
dl_train = DataLoader(
    dataset=ds_train, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=collate_fn
)

# Model Design

## Execution Flow
1. Convert all characters in the sentence into embeddings.
2. Pass the embeddings through an LSTM sequentially.
3. The output of the LSTM is passed into another LSTM, and additional layers can be added.
4. The output from all time steps of the final LSTM is passed through a Fully Connected layer.
5. The character corresponding to the maximum value across all output dimensions is selected as the next character.

## Loss Function
Since this is a classification task, Cross Entropy is used as the loss function.

## Gradient Update
Adam algorithm is used for gradient updates.

In [ ]:
class CharRNN(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(CharRNN, self).__init__()
        
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size,
                                            embedding_dim=embed_dim,
                                            padding_idx=char_to_id['<pad>'])
        
        self.rnn_layer1 = torch.nn.LSTM(input_size=embed_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.rnn_layer2 = torch.nn.LSTM(input_size=hidden_dim,
                                        hidden_size=hidden_dim,
                                        batch_first=True)
        
        self.linear = torch.nn.Sequential(torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=hidden_dim),
                                          torch.nn.ReLU(),
                                          torch.nn.Linear(in_features=hidden_dim,
                                                          out_features=vocab_size))
        
    def forward(self, batch_x, batch_x_lens):
        return self.encoder(batch_x, batch_x_lens)
    
    # The forward pass of the model
    def encoder(self, batch_x, batch_x_lens):
        batch_x = self.embedding(batch_x)
        
        batch_x = torch.nn.utils.rnn.pack_padded_sequence(batch_x,
                                                          batch_x_lens,
                                                          batch_first=True,
                                                          enforce_sorted=False)
        
        batch_x, _ = self.rnn_layer1(batch_x)
        batch_x, _ = self.rnn_layer2(batch_x)
        
        batch_x, _ = torch.nn.utils.rnn.pad_packed_sequence(batch_x,
                                                            batch_first=True)
        
        batch_x = self.linear(batch_x)
        
        return batch_x
    
    def generator(self, start_char, max_len=200):
        
        char_list = [char_to_id[c] for c in start_char]
        answer_list = []
        
        next_char = None
        
        while len(char_list) < max_len: 
            # Write your code here 
            # Pack the char_list to tensor
            input_tensor = torch.tensor([char_list], dtype=torch.long, device=self.embedding.weight.device)
            input_lens = torch.tensor([len(char_list)], dtype=torch.long)
            # Input the tensor to the embedding layer, LSTM layers, linear respectively
            logits = self.encoder(input_tensor, input_lens)
            # Obtain the next token prediction y
            y = logits[0, -1, :]
            
            next_char = torch.argmax(y, dim=-1).item() # Use argmax function to get the next token prediction
            
            char_list.append(next_char)
            
            if next_char == char_to_id['<eos>']:
                break
            
            
        return [id_to_char[ch_id] for ch_id in char_list]

In [15]:
torch.manual_seed(2)


device = 'cuda' # torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu') # Write your code here. Specify a device (cuda or cpu)

model = CharRNN(vocab_size,
                embed_dim,
                hidden_dim)

In [16]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=char_to_id['<pad>']) # Write your code here. Cross-entropy loss function. The loss function should ignore <pad>
optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Write your code here. Use Adam or AdamW for Optimizer

# Training
1. The outer `for` loop controls the `epoch`
    1. The inner `for` loop uses `data_loader` to retrieve batches.
        1. Pass the batch to the `model` for training.
        2. Compare the predicted results `batch_pred_y` with the true labels `batch_y` using Cross Entropy to calculate the loss `loss`
        3. Use `loss.backward` to automatically compute the gradients.
        4. Use `torch.nn.utils.clip_grad_value_` to limit the gradient values between `-grad_clip` &lt; and &lt; `grad_clip`.
        5. Use `optimizer.step()` to update the model (backpropagation).
2.  After every `1000` batches, output the current loss to monitor whether it is converging.

In [17]:
from tqdm import tqdm
from copy import deepcopy
model = model.to(device)
model.train()
i = 0
for epoch in range(1, epochs+1):
    # The process bar
    bar = tqdm(dl_train, desc=f"Train epoch {epoch}")
    for batch_x, batch_y, batch_x_lens, batch_y_lens in bar:
        # Write your code here
        # Clear the gradient
        optimizer.zero_grad()
        # Input the data to the model
        batch_pred_y = model(batch_x.to(device), batch_x_lens)
        
        # Write your code here
        # Input the prediction and ground truths to loss function
        loss = criterion(batch_pred_y.view(-1, vocab_size), batch_y.to(device).view(-1))
        # Back propagation
        loss.backward()

        torch.nn.utils.clip_grad_value_(model.parameters(), grad_clip) # gradient clipping

        # Write your code here
        # Optimize parameters in the model
        optimizer.step()

        i+=1
        if i%50==0:
            bar.set_postfix(loss = loss.item())
    
    # Evaluate your model
    matched = 0
    total = 0
    bar_eval = tqdm(df_eval.iterrows(), desc=f"Validation epoch {epoch}")
    for _, row in bar_eval:
        batch_x = row['src']
        batch_y = row['tgt']

        # An example of using generator: model.generator('1+1=')
        
        # Write your code here. Input the batch_x to the model and generate the predictions
        prediction = model.generator(batch_x)
        if '=' in row['src']:
            equal_idx = row['src'].index('=')
        else:
            equal_idx = len(row['src'])
        pred_answer_list = prediction[equal_idx + 1:]
        # Remove the <eos> token if present
        if '<eos>' in pred_answer_list:
            pred_answer_list.remove('<eos>')
        # Reconstruct the predicted answer string

        # Write your code here.
        # Check whether the prediction match the ground truths
        pred_answer = "".join(pred_answer_list)
        if pred_answer == batch_y:
            matched += 1
        total += 1

        # Compute exact match (EM) on the eval dataset
        i+=1
        if i%50==0:
            bar_eval.set_postfix(val_accuracy = "{:.2f} ({}/{})".format(matched/total, matched, total))
        # EM = correct/total

    print(matched/total)

Train epoch 1: 100%|██████████| 37020/37020 [05:53<00:00, 104.59it/s, loss=0.285]
Validation epoch 1: 263250it [18:40, 234.96it/s, val_accuracy=0.56 (146956/263230)]


0.558252611585945


Train epoch 2: 100%|██████████| 37020/37020 [05:56<00:00, 103.99it/s, loss=0.325]
Validation epoch 2: 263250it [18:44, 234.15it/s, val_accuracy=0.64 (167851/263210)]

0.6376866096866097
